# Part I. ETL Pipeline for Pre-Processing the Files

## CODE FOR PRE-PROCESSING THE FILES

#### Import Python packages 

In [1]:
# Import Python packages 
import pandas as pd
import cassandra
import re
import os
import glob
import numpy as np
import json
import csv

In [2]:
rm -rf /home/workspace/*/.ipynb_checkpoints

#### Creating list of filepaths to process original event csv data files

In [3]:
# checking current working directory
print(os.getcwd())

# Getting current folder and subfolder event data
filepath = os.getcwd() + '/event_data'
print(filepath)

# Create a for loop to create a list of files and collect each filepath
for root, dirs, files in os.walk(filepath):
    
# join the file path and roots with the subdirectories using glob
    file_path_list = glob.glob(os.path.join(root,'*'))
#     print(file_path_list)

/home/workspace
/home/workspace/event_data


#### Processing the files to create the data file csv that will be used for Apache Casssandra tables

In [4]:
# initiating an empty list of rows that will be generated from each file
full_data_rows_list = [] 
    
# for every filepath in the file path list 
for f in file_path_list:

# reading csv file 
    with open(f, 'r', encoding = 'utf8', newline='') as csvfile: 
        # creating a csv reader object 
        csvreader = csv.reader(csvfile) 
        next(csvreader)
        
 # extracting each data row one by one and append it        
        for line in csvreader:
            # print(line)
            full_data_rows_list.append(line) 
            
# uncomment the code below to get total number of rows 
# print(len(full_data_rows_list))
# uncomment the code below to see what the list of event data rows will look like
# print(full_data_rows_list)

# creating a smaller event data csv file called event_datafile_full csv that will be used to insert data into the \
# Apache Cassandra tables
csv.register_dialect('myDialect', quoting=csv.QUOTE_ALL, skipinitialspace=True)

with open('event_datafile_new.csv', 'w', encoding = 'utf8', newline='') as f:
    writer = csv.writer(f, dialect='myDialect')
    writer.writerow(['artist','firstName','gender','itemInSession','lastName','length',\
                'level','location','sessionId','song','userId'])
    for row in full_data_rows_list:
        if (row[0] == ''):
            continue
        writer.writerow((row[0], row[2], row[3], row[4], row[5], row[6], row[7], row[8], row[12], row[13], row[16]))


In [5]:
# check the number of rows in the csv file
with open('event_datafile_new.csv', 'r', encoding = 'utf8') as f:
    print(sum(1 for line in f))

6821


# Part II. Apache Cassandra coding portion of the project

## Now we are ready to work with the CSV file titled <font color=red>event_datafile_new.csv</font>, located within the Workspace directory.  The event_datafile_new.csv contains the following columns: 
- artist 
- firstName of user
- gender of user
- item number in session
- last name of user
- length of the song
- level (paid or free song)
- location of the user
- sessionId
- song title
- userId

The image below is a screenshot of what the denormalized data should appear like in the <font color=red>**event_datafile_new.csv**</font> after the code above is run:<br>

<img src="images/image_event_datafile_new.jpg">

## Apache Cassandra code:

#### Creating a Cluster

In [6]:
# This should make a connection to a Cassandra instance in my local machine 
# (127.0.0.1)

from cassandra.cluster import Cluster
cluster = Cluster()

# To establish connection and begin executing queries, need a session
session = cluster.connect()

#### Create Keyspace

In [7]:
# Create a Keyspace named as song_database
keysp = '''CREATE KEYSPACE IF NOT EXISTS song_database
                WITH REPLICATION = {
                    'class': 'SimpleStrategy',
                    'replication_factor': 1       
                    }'''

In [8]:
session.execute(keysp)

#### Set Keyspace

In [9]:
# Set KEYSPACE to the keyspace specified above
session.set_keyspace('song_database')

### Now we need to create tables to run the following queries. With Apache Cassandra we model the database tables on the queries we want to run.

### Queries will be based on these three questions:

##### 1. Give me the artist, song title and song's length in the music app history that was heard during  sessionId = 338, and itemInSession  = 4


##### 2. Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182
    

##### 3. Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'

#### Let's get started:

#### Query 1:  Give me the artist, song title and song's length in the music app history that was heard during sessionId = 338, and itemInSession = 4

##### Data Modeling Design Explanation:
##### I am calling the table for this query as 'song_info' as this query is asking for the information of songs and I have chosen item_in_session as partitioning key (each row is partitioned by the item_in_session) and session_id as clustering key (each row is then sorted by the session_id)

In [10]:
create_table1 = """CREATE TABLE IF NOT EXISTS song_info(
                    item_in_session int,
                    session_id int,
                    artist_name varchar,
                    song_length float,
                    song_title varchar,
                    PRIMARY KEY(item_in_session, session_id)
                    )"""
session.execute(create_table1);

In [11]:
# We have provided part of the code to set up the CSV file.
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
    ## Assign the INSERT statements into the `query` variable
        query = """INSERT INTO song_info(
                        artist_name,
                        item_in_session,
                        song_length,
                        session_id,
                        song_title
                        )"""
        query = query + "VALUES (%s, %s, %s, %s, %s)"
        ## Assign which column element should be assigned for each column in the INSERT statement.
        ## For e.g., to INSERT artist_name and user first_name, change the code below to `line[0], line[1]`
        session.execute(query, (line[0], int(line[3]), float(line[5]), int(line[8]), line[9]));

##### lets do a SELECT to verify that the data have been inserted into table

In [12]:
## Add in the SELECT statement to verify the data was entered into the table
query1 = "SELECT artist_name, song_length, song_title FROM song_info WHERE item_in_session = 4 AND session_id = 338"
out_query1 = session.execute(query1)

for row in out_query1:
    print(row.artist_name, row.song_length, row.song_title)

Faithless 495.30731201171875 Music Matters (Mark Knight Dub)


#### Query 2: Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name)for userid = 10, sessionid = 182

##### Data Modeling Design Explanation:
##### I am calling the table for this query as 'songplay_info' as this query is asking for the information of songs with artist names played by the users and I have chosen user_id and session_id as partitioning key (each row is first partitioned by the user_id and session_id) and item_in_session as clustering key (each row is then sorted by the item_in_session)

In [13]:
create_table2 = """CREATE TABLE songplay_info(
                        user_id int,
                        session_id int,
                        item_in_session int,
                        artist_name varchar,
                        first_name varchar,
                        last_name varchar,
                        song_title varchar,
                        PRIMARY KEY((user_id, session_id), item_in_session)
                        )"""
session.execute(create_table2);                    

In [14]:
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
    ## Assign the INSERT statements into the `query` variable
        query = """INSERT INTO songplay_info(
                        user_id,
                        session_id,
                        item_in_session,
                        artist_name,
                        first_name,
                        last_name,
                        song_title
                        )"""
        query = query + "VALUES (%s, %s, %s, %s, %s, %s, %s)"
        ## Assign which column element should be assigned for each column in the INSERT statement.
        ## For e.g., to INSERT artist_name and user first_name, change the code below to `line[0], line[1]`
        session.execute(query, (int(line[10]), int(line[8]), int(line[3]), line[0], line[1], line[4], line[9]));

##### lets do a SELECT to verify that the data have been inserted into table

In [15]:
query2 = "SELECT artist_name, song_title, first_name FROM songplay_info WHERE user_id = 10 AND session_id = 182"
out_query2 = session.execute(query2);

for row in out_query2:
    print(row.artist_name, row.song_title, row.first_name)

Down To The Bone Keep On Keepin' On Sylvie
Three Drives Greece 2000 Sylvie
Sebastien Tellier Kilometer Sylvie
Lonnie Gordon Catch You Baby (Steve Pitron & Max Sanna Radio Edit) Sylvie


##### Query 3: Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'

##### Data Modeling Design Explanation:
##### I am calling the table for this query as 'user_info' as this query is asking for the information of users only and I have chosen song_title as partitioning key (each row is first partitioned by the song_title) and user_id as clustering key (each row is then sorted by the user_id)

In [16]:
create_table3 = """CREATE TABLE user_info(
                        song_title varchar,
                        user_id int,
                        first_name varchar,
                        last_name varchar,
                        PRIMARY KEY(song_title, user_id)
                        )"""
session.execute(create_table3);                     

In [17]:
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
    ## Assign the INSERT statements into the `query` variable
        query = """INSERT INTO user_info(
                        song_title,
                        user_id,
                        first_name,
                        last_name
                        )"""
        query = query + "VALUES (%s, %s, %s, %s)"
        ## Assign which column element should be assigned for each column in the INSERT statement.
        ## For e.g., to INSERT artist_name and user first_name, change the code below to `line[0], line[1]`
        session.execute(query, (line[9], int(line[10]), line[1], line[4]));

##### lets do a SELECT to verify that the data have been inserted into table

In [18]:
query3 = "SELECT first_name, last_name FROM user_info WHERE song_title = 'All Hands Against His Own'"
out_query3 = session.execute(query3);

for row in out_query3:
    print(row.first_name, row.last_name)

Jacqueline Lynch
Tegan Levine
Sara Johnson


### Drop the tables before closing out the sessions

In [19]:
query = "DROP TABLE IF EXISTS song_info"
try:
    session.execute(query)
except Exception as e:
    print(e)

query = "DROP TABLE IF EXISTS songplay_info"
try:
    session.execute(query)
except Exception as e:
    print(e)

query = "DROP TABLE IF EXISTS user_info"
try:
    session.execute(query)
except Exception as e:
    print(e);

### Close the session and cluster connection¶

In [20]:
session.shutdown()
cluster.shutdown()